<a href="https://colab.research.google.com/github/B-Lorentz/physics-bsc-visual/blob/main/3_Exact_solutions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

import numpy as np
from matplotlib import pyplot as plt
from jax import numpy as jnp
from jax import grad, vmap, jit, random, lax, ops
from matplotlib.widgets import Slider, Button, RadioButtons
from ipywidgets import interact, widgets